In [ ]:
# ========================================================
# MODULE 3: REAL-TIME FRAUD DETECTION ENGINE (Pydantic V2)
# ========================================================

import pandas as pd
import joblib
import numpy as np
import onnxruntime as ort
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn
from datetime import datetime
import os
import requests
from IPython.display import display, HTML
import json
import threading
import time

# -------------------- PATHS --------------------
BASE_DIR = r"C:\Users\ghosh\Desktop\Predictive Transaction Intelligence using for BFSI\api"
PREPROC_PATH = os.path.join(BASE_DIR, "preprocessor.pkl")
MODEL_PATH = os.path.join(BASE_DIR, "fraud_model.onnx")

os.makedirs(BASE_DIR, exist_ok=True)

# -------------------- LOAD MODEL & PREPROCESSOR --------------------
print("Loading model and preprocessor...")
try:
    preprocessor = joblib.load(PREPROC_PATH)
    session = ort.InferenceSession(MODEL_PATH)
    input_name = session.get_inputs()[0].name
    print("Model loaded!")
except Exception as e:
    raise FileNotFoundError(f"Check paths:\n{PREPROC_PATH}\n{MODEL_PATH}\nError: {e}")

# -------------------- FASTAPI APP --------------------
app = FastAPI(title="Real-Time Fraud Engine", version="3.0")

# -------------------- FRAUD SIGNATURES --------------------
FRAUD_SIGNATURES = {
    "high_amount": 50_000_000,
    "foreign_country": ["Russia", "Turkey", "USA", "China", "UAE"],
    "velocity_threshold": 10,
    "night_transaction": (0, 5),
}

USER_BEHAVIOR = {}

# -------------------- SCHEMAS --------------------
class Transaction(BaseModel):
    User_ID: int
    Transaction_Amount: float
    Transaction_Location: str
    Merchant_ID: int
    Device_ID: int
    Card_Type: str
    Transaction_Currency: str
    Transaction_Status: str
    Previous_Transaction_Count: int
    Distance_Between_Transactions_km: float
    Time_Since_Last_Transaction_min: int
    Authentication_Method: str
    Transaction_Velocity: int
    Transaction_Category: str
    Transaction_Hour: int
    Transaction_Day: int
    Transaction_Month: int
    Transaction_Weekday: int
    Log_Transaction_Amount: float
    Velocity_Distance_Interact: float
    Amount_Velocity_Interact: float
    Time_Distance_Interact: float
    Hour_sin: float
    Hour_cos: float
    Weekday_sin: float
    Weekday_cos: float

class AlertResponse(BaseModel):
    Transaction_ID: int
    User_ID: int
    Fraud_Probability: float
    Final_Risk_Score: float
    isFraud_pred: int
    alert_triggered: bool
    alert_reasons: list
    timestamp: str

# -------------------- ALERT DISPLAY --------------------
def send_alert(reasons: list, tx: dict):
    tx_id = int(datetime.now().timestamp() * 1000)
    msg = f"""
    <div style="background:#ffebee;padding:15px;border-left:6px solid #f44336;font-family:Arial">
    <h3>REAL-TIME FRAUD ALERT</h3>
    <b>User ID:</b> {tx['User_ID']}<br>
    <b>Amount:</b> {tx['Transaction_Amount']:,.0f} UZS<br>
    <b>Location:</b> {tx['Transaction_Location']}<br>
    <b>Time:</b> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}<br>
    <b>Reasons:</b> {', '.join(reasons)}<br>
    <b>Tx ID:</b> {tx_id}
    </div>
    """
    display(HTML(msg))

# -------------------- RISK ENGINE --------------------
def evaluate_risk(tx: dict) -> dict:
    df = pd.DataFrame([tx])
    tx_id = int(datetime.now().timestamp() * 1000)
    df.insert(0, 'Transaction_ID', tx_id)

    # Model Prediction
    try:
        X = preprocessor.transform(df.drop(columns=['Transaction_ID', 'User_ID'], errors='ignore'))
        if hasattr(X, 'toarray'):
            X = X.toarray().astype(np.float32)
        prob = float(session.run(None, {input_name: X})[0][0][0])
    except:
        prob = 0.5

    model_pred = int(prob > 0.5)
    reasons = []
    risk_boost = 0.0

    if tx['Transaction_Amount'] > FRAUD_SIGNATURES['high_amount']:
        reasons.append("High Amount"); risk_boost += 0.35
    if FRAUD_SIGNATURES['night_transaction'][0] <= tx['Transaction_Hour'] < FRAUD_SIGNATURES['night_transaction'][1]:
        reasons.append("Night Transaction"); risk_boost += 0.25
    if tx['Transaction_Velocity'] > FRAUD_SIGNATURES['velocity_threshold']:
        reasons.append("High Velocity"); risk_boost += 0.20
    if tx['Transaction_Location'] in FRAUD_SIGNATURES['foreign_country']:
        reasons.append("Foreign Location"); risk_boost += 0.25

    # Behavioral: New Device
    user_id = tx['User_ID']
    if user_id not in USER_BEHAVIOR:
        USER_BEHAVIOR[user_id] = {'devices': set(), 'tx_count_1h': 0, 'last_hour': datetime.now().hour}
    behavior = USER_BEHAVIOR[user_id]

    if tx['Device_ID'] not in behavior['devices']:
        reasons.append("New Device"); risk_boost += 0.30
        behavior['devices'].add(tx['Device_ID'])

    # Activity burst
    current_hour = datetime.now().hour
    if current_hour != behavior['last_hour']:
        behavior['tx_count_1h'] = 0
        behavior['last_hour'] = current_hour
    behavior['tx_count_1h'] += 1
    if behavior['tx_count_1h'] > 8:
        reasons.append("Activity Burst"); risk_boost += 0.20

    final_risk = min(prob + risk_boost, 1.0)
    alert = final_risk > 0.7 or model_pred == 1

    if alert and reasons:
        send_alert(reasons, tx)

    return {
        "Transaction_ID": tx_id,
        "User_ID": user_id,
        "Fraud_Probability": round(prob, 4),
        "Final_Risk_Score": round(final_risk, 4),
        "isFraud_pred": model_pred,
        "alert_triggered": alert,
        "alert_reasons": reasons,
        "timestamp": datetime.now().isoformat()
    }

# -------------------- ENDPOINTS --------------------
@app.post("/detect", response_model=AlertResponse)
def detect_fraud(transaction: Transaction):
    try:
        # FIXED: Use model_dump() instead of dict()
        result = evaluate_risk(transaction.model_dump())
        return result
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
def home():
    return {"module": "3", "status": "ACTIVE", "docs": "http://127.0.0.1:8000/docs"}

# -------------------- START SERVER --------------------
print("\nStarting server...")
def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="error")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
time.sleep(2)

# -------------------- TEST --------------------
print("Testing high-risk transaction...")
test_tx = {
    "User_ID": 99999,
    "Transaction_Amount": 75_000_000,
    "Transaction_Location": "Russia",
    "Merchant_ID": 1234,
    "Device_ID": 999999,
    "Card_Type": "UzCard",
    "Transaction_Currency": "UZS",
    "Transaction_Status": "Successful",
    "Previous_Transaction_Count": 1,
    "Distance_Between_Transactions_km": 5000,
    "Time_Since_Last_Transaction_min": 2,
    "Authentication_Method": "Password",
    "Transaction_Velocity": 15,
    "Transaction_Category": "Transfer",
    "Transaction_Hour": 2,
    "Transaction_Day": 28,
    "Transaction_Month": 10,
    "Transaction_Weekday": 1,
    "Log_Transaction_Amount": 18.2,
    "Velocity_Distance_Interact": 75000,
    "Amount_Velocity_Interact": 1125000000,
    "Time_Distance_Interact": 10000,
    "Hour_sin": 0.0,
    "Hour_cos": 1.0,
    "Weekday_sin": 0.0,
    "Weekday_cos": 1.0
}

try:
    resp = requests.post("http://127.0.0.1:8000/detect", json=test_tx)
    result = resp.json()
    print("\nResponse:")
    print(json.dumps(result, indent=2))
except:
    print("Server not ready. Wait and retry.")

print("\nMODULE 3 IS LIVE!")
print("Swagger UI: http://127.0.0.1:8000/docs")

Loading model and preprocessor...
Model loaded!

Starting server...


ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted


Testing high-risk transaction...


C:\Users\ghosh\AppData\Local\Temp\ipykernel_24568\2437478716.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = evaluate_risk(transaction.dict())



Response:
{
  "Transaction_ID": 1761671141801,
  "User_ID": 99999,
  "Fraud_Probability": 0.5,
  "Final_Risk_Score": 1.0,
  "isFraud_pred": 0,
  "alert_triggered": true,
  "alert_reasons": [
    "High Amount",
    "Night Transaction",
    "High Velocity",
    "Foreign Location",
    "New Device"
  ],
  "timestamp": "2025-10-28T22:35:41.813039"
}

MODULE 3 IS LIVE!
Swagger UI: http://127.0.0.1:8000/docs


C:\Users\ghosh\AppData\Local\Temp\ipykernel_24568\2437478716.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = evaluate_risk(transaction.dict())


C:\Users\ghosh\AppData\Local\Temp\ipykernel_24568\2437478716.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = evaluate_risk(transaction.dict())


C:\Users\ghosh\AppData\Local\Temp\ipykernel_24568\2437478716.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = evaluate_risk(transaction.dict())
C:\Users\ghosh\AppData\Local\Temp\ipykernel_24568\2437478716.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = evaluate_risk(transaction.dict())
C:\Users\ghosh\AppData\Local\Temp\ipykernel_24568\2437478716.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = evaluate_risk(transaction.dict())
C:\Users\g

C:\Users\ghosh\AppData\Local\Temp\ipykernel_24568\2437478716.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = evaluate_risk(transaction.dict())
C:\Users\ghosh\AppData\Local\Temp\ipykernel_24568\2437478716.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  result = evaluate_risk(transaction.dict())
